In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Layer
from tensorflow.keras import layers
from tensorflow.keras.initializers import Constant
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

import cv2
import os

import numpy as np
import pandas as pd
import math
import random

In [ ]:
path = '../input/petfinder-pawpularity-score/'
scores = pd.read_csv(path + 'train.csv')
img_size=340
current_batch = 0
last_batch = 0
batchrates = []

rlrop = ReduceLROnPlateau(monitor='root_mean_squared_error', factor=0.5, patience=5, verbose=1)

def get_batch_rate(epoch, lrate):
    global batchrates
    global current_batch
    global last_batch
    batchrates[last_batch] = lrate
    last_batch = current_batch 
    lrate = batchrates[current_batch]
    return lrate

lrs = LearningRateScheduler(get_batch_rate)

def root_mean_squared_error(y_true, y_pred):
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square(y_pred - y_true)))

def preprocessdata(X):
    X_p = keras.applications.resnet50.preprocess_input(X)
    return X_p

def shuffle_batch(x, y):
    new_x = []
    new_y = []
    r = []
    for z in range(len(x)):
        r.append(z)
    random.shuffle(r)
    for z in range(len(x)):
        new_x.append(x[r[z]])
        new_y.append(y[r[z]])
    
    del x
    del y
    return new_x, new_y
        

def assemble_batch(batch_num, batch_length):
    start_index = batch_num * batch_length
    total_length = scores.shape[0]
    data = []
    if (batch_num == batches - 1):
        if (total_length % batches > 0):
            batch_length = batch_length + (total_length % batches)
    end_index = start_index + batch_length
    for x in range(start_index, end_index):
        entry = scores.iloc[x]
        try:
            img_arr = cv2.imread(os.path.join(path + 'train', entry.Id + '.jpg'))[...,::-1] 
            resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
            data.append([resized_arr, entry])
        except Exception as e:
            print(e)
    print(f'Batch {batch_num+1} assembled. Length: {batch_length}')
    return np.array(data, dtype=object)
    
def train_batch(z, model, batch_length, max_epochs, lastLoss=999, rMetric="loss", f=1, d=0.0, q=1.0): 
    global batchrates
    global current_batch
    
    train = assemble_batch(z, batch_length)
    current_batch = z
    x_train = []
    y_train = []

    for feature, labels in train:
        x_train.append(feature)
        y_train.append(labels[f:])

    del train    
    x_train, y_train = shuffle_batch(x_train, y_train)
        
    for x in range(len(y_train)):
        y_train[x] = y_train[x].to_list()
        for y in range(len(y_train[x])):
            if (y == len(y_train[x]) - 1):
                y_train[x][y] = (float(y_train[x][y]) / q) + d
            y_train[x][y] = float(y_train[x][y])
        y_train[x] = np.array(y_train[x])
        
    x_train = np.array(x_train) / 255
    #x_train.reshape(-1, img_size, img_size, 1)
    x_train = preprocessdata(x_train)
    
    y_train = np.asarray(y_train).astype('float32')
    
    t_model = model
    e = max_epochs
    passing = 0
    
    while (passing == 0):
        history = model.fit(x_train,y_train,epochs = e,batch_size=2,callbacks=[rlrop,lrs])
        l_hist = history.history[rMetric]     
        trend = 0
        fLoss = lastLoss
        for x in range(len(l_hist)):
            if (x > 0):
                if (l_hist[x] < lastLoss):
                    trend = trend - 1
                else:
                    trend = trend + 1
            lastLoss = l_hist[x]
                    
        if (trend >= 0):
            if (l_hist[0] >= l_hist[len(l_hist) - 1]):
                e = max_epochs
                passing = 1
            else:
                if (e > 1):
                    e = e - 1
                    batchrates[z] = batchrates[z] / 2
                    keras.backend.set_value(model.optimizer.learning_rate,  batchrates[z])
                    model = t_model
                    lastLoss = fLoss
                else:
                    e = max_epochs
                    passing = 1
        else:
            e = max_epochs
            passing = 1
        
    del x_train
    del y_train
    del history
    del l_hist
    del t_model
    return model, lastLoss

In [ ]:
res_model = keras.applications.ResNet50(include_top=False, weights="../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5", input_tensor=keras.Input(shape=(img_size, img_size, 3)))

In [ ]:
for layer in res_model.layers[:143]:
    layer.trainable = False

In [ ]:
model1 = keras.Sequential([
    layers.Lambda(lambda image: tf.image.resize(image, (img_size, img_size))),
    res_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'), #Initial Transfer Layer
    layers.Dense(128, activation='relu'),  #==|
    layers.Dense(64, activation='relu'),   #  |- Linear Units 
    layers.Dense(32, activation='relu'),   #==|
    layers.Dense(13, activation='relu') #Transition Layer
])

In [ ]:
model1.compile(loss=tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE), optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5), metrics=['RootMeanSquaredError', 'binary_accuracy'])

In [ ]:
batches = 10
batch_length = math.floor(scores.shape[0] / batches)
batchrates = np.full((batches), 2e-5)
print(f'Total Length: {scores.shape[0]} | Batches: {batches} | Batch Length: {batch_length}')

In [ ]:
#Stage 1: Induction
lastLoss = 999

for z in range(batches):
    print(f'Starting Batch {z+1} out of {batches}')
    print(f'Batch Rate: {batchrates[z]}')
    model1, lastLoss = train_batch(z, model1, batch_length, 10, lastLoss, d=0.25, q=100.0)

In [ ]:
for layer in model1.layers[:-4]:
    layer.trainable = False

In [ ]:
#Stage 2: Focus
lastLoss = 999
overlap = 5
batchrates = np.full((batches), 2e-5)
keras.backend.set_value(model1.optimizer.learning_rate,  2e-5)
for x in range(2, batches + overlap):
    y = 0
    if (x > overlap):
        y = x - overlap
    if (x > batches):
        x = batches
    for z in range(y, x):
        print(f'Starting Batch {z+1} out of {batches}')
        print(f'Batch Rate: {batchrates[z]}')
        model1, lastLoss = train_batch(z, model1, batch_length, 5, lastLoss, d=0.25, q=100.0)


In [ ]:
for layer in model1.layers:
    layer.trainable = False

In [ ]:
model2 = keras.Sequential([
    model1,
    keras.layers.Dense(10, activation='relu'), #Transfer Layer
    keras.layers.Dense(1, activation='relu') #Output
])

In [ ]:
model2.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5) , loss = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.NONE) , metrics = ['RootMeanSquaredError'])

In [ ]:
#Stage 3: Funneling
lastLoss = 999
overlap = 3
batchrates = np.full((batches), 2e-5)
for x in range(2, batches + overlap):
    y = 0
    if (x > overlap):
        y = x - overlap
    if (x > batches):
        x = batches
    for z in range(y, x):
        print(f'Starting Batch {z+1} out of {batches}')
        print(f'Batch Rate: {batchrates[z]}')
        model2, lastLoss = train_batch(z, model2, batch_length, 10, lastLoss, f=-1, q=100.0)

In [ ]:
#Stage 4: Repeatability
lastLoss = 999
batchrates = np.full((batches), 2e-5)
keras.backend.set_value(model1.optimizer.learning_rate,  2e-5)
for z in range(batches):
    print(f'Starting Batch {z+1} out of {batches}')
    print(f'Batch Rate: {batchrates[z]}')
    model2, lastLoss = train_batch(z, model2, batch_length, 3, lastLoss, f=-1, q=100.0)

In [ ]:
test_data = []
test_results = []
test_ids = []

for filename in os.listdir(path + 'test'):
    try:
        test_ids.append(filename[:-4])
        img_arr = cv2.imread(os.path.join(path + 'test', filename))[...,::-1] 
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) 
        test_data.append(resized_arr)
        test_data = np.array(test_data) / 255
        #test_data.reshape(-1, img_size, img_size, 1)
        test_data = preprocessdata(test_data)
        test_results.append(model2.predict(test_data).tolist()[0])
        test_data = []
        
    except Exception as e:
        print(e)
        
for x in range(len(test_results)):
    test_results[x] = test_results[x][0] * 100
print(test_results)
print(test_ids)

In [ ]:
output_dict = {'Id': test_ids, 'Pawpularity': test_results}
output_df = pd.DataFrame(output_dict)
output_df.head()

In [ ]:
output_df.to_csv("submission.csv",index=False)